In [ ]:
import trimesh
import os
import torch
import pickle
import numpy as np

obj2idx = {
"book" : 1,
"espresso" : 2,
"lotion" : 3,
"lotion_spray" : 4,
"milk" : 5,
"cocoa" : 6,
"chips" : 7,
"cappuccino" : 8,
}
idx2obj = {v:k for k, v in obj2idx.items()}

sample_points = 2000
obj_path = {k : os.path.join('/HDD/hoseong/H2O', 'object', v, f'{v}.obj') for k, v in idx2obj.items()}
obj_info = {k : trimesh.load(path) for k, path in obj_path.items()}
obj_info = {k : trimesh.Trimesh(obj.vertices, obj.faces) for k, obj in obj_info.items()}
obj_vertices = {k: trimesh.sample.sample_surface(obj, sample_points)[0]  for k, obj in obj_info.items()}
gt_3D_bbox = {}
for i in range(8):
    with open(f'/HDD/hoseong/H2O/subject3/h1/{i}/cam4/obj_pose/000000.txt', 'r') as f:
        file = f.read().split(' ')
        label = int(file[0])
        gt_obj_pose = torch.Tensor(list(map(float, file[1:-1]))).reshape(21,3)
    with open(f'/HDD/hoseong/H2O/subject3/h1/{i}/cam4/obj_pose_rt/000000.txt', 'r') as f:
        gt_obj_rt = torch.Tensor(list(map(float, f.read().split(' ')[1:-1]))).reshape(4,4)
    gt_obj_pose = torch.cat([gt_obj_pose, torch.ones(21,1)], dim=-1)
    gt_obj_pose = torch.inverse(gt_obj_rt).matmul(gt_obj_pose.T).T[:,:-1]
    gt_3D_bbox[label] = gt_obj_pose

In [ ]:
for i in range(1,9):
    with open(os.path.join('/HDD/hoseong/H2O', 'obj_pkl', f'{idx2obj[i]}_{sample_points}.pkl'), 'wb') as f:
        pickle.dump(np.array(obj_vertices[i]), f)